In [ ]:
!pip install gradio langchain_groq langchain-community huggingface_hub PyPDF2 langchain-huggingface faiss-cpu

In [ ]:
from dotenv import load_dotenv
from huggingface_hub import login
from PyPDF2 import PdfReader
import gradio as gr
import os

from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [ ]:
# Load env vars (if any) and Hugging Face API token
load_dotenv()
login(os.getenv('HUGGINGFACEHUB_API_TOKEN'))


In [ ]:
# STEP 3: Upload and process PDF (can replace with fixed path)
from google.colab import files
uploaded = files.upload()
pdf_paths = list(uploaded.keys())

Saving policy.pdf to policy (2).pdf


In [ ]:
# Step 4: Extract text
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
    return text

raw_text = get_pdf_text(pdf_paths)

In [ ]:
# Step 5: Split text into chunks
def get_text_chunks(raw_text):
    splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=200)
    return splitter.split_text(raw_text)

text_chunks = get_text_chunks(raw_text)

In [ ]:
# Step 6: Create vector store
def get_vectorstore(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="hkunlp/instructor-xl", model_kwargs={"device":"cpu"})
    return FAISS.from_texts(texts=chunks, embedding=embeddings)

vectorstore = get_vectorstore(text_chunks)

In [ ]:
# Step 7: Setup chatbot with memory
def get_conversation_chain(vstore):
    llm = ChatGroq(
        model_name="llama3-8b-8192",
        temperature=0.7,
        request_timeout=30,
        api_key="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"  # Replace this securely
    )
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    return ConversationalRetrievalChain.from_llm(llm=llm, retriever=vstore.as_retriever(), memory=memory)

conversation = get_conversation_chain(vectorstore)

/tmp/ipython-input-24-1301550106.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [ ]:
# Step 8: Define chatbot function
def chat_bot(user_input, history=[]):
    result = conversation({"question": user_input})
    reply = result["answer"]
    history.append((user_input, reply))
    return history, history




In [ ]:
# Step 9: Launch Gradio API
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    message = gr.Textbox(label="Ask a question about the policy")
    clear = gr.Button("Clear chat")

    message.submit(chat_bot, [message, chatbot], [chatbot, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(share=True)


/tmp/ipython-input-26-2006372929.py:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5791e6cb331a909895.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
chat_with_pdf("When the education policy was introduced ?")


In [ ]:
!pip install gradio


In [ ]:
import gradio as gr

# Placeholder: Replace this with your actual policy handling logic
def chat_bot(message, history=[]):
    # You can replace this with logic from your notebook (LLMs, LangChain, etc.)
    response = f"Policy bot answer to: {message}"
    history.append(("You: " + message, "Bot: " + response))
    return history, history



In [ ]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Policy Bot")
    msg = gr.Textbox(label="Ask about a policy")
    clear = gr.Button("Clear")

    def respond(message, history):
        return chat_bot(message, history)

    msg.submit(respond, [msg, chatbot], [chatbot, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(share=True)  # This gives a public URL


To fix the `AuthenticationError`, you need to provide a valid Groq API key. It's recommended to store your API keys securely as Colab Secrets.

1. Click on the "🔑" icon in the left sidebar.
2. Click "Add new secret".
3. For the **Name**, use `GROQ_API_KEY`.
4. For the **Value**, paste your Groq API key.
5. Make sure the "Notebook access" is toggled ON for this notebook.

Once you've added the secret, I'll modify the code to use it.